Парсим исторические сведения о погоде с июля 2017 года по апрель 2023 с сайта https://pogoda1.ru/ для Москвы (точнее - метеостанции Внуково) и Санкт-Петербурга (точнее - метеостанции Пулково).

Очень приятный сайт, для которого не нужно заморачиваться с fake useragent и запросами через прокси. 

Нужные страницы лежат под простой маской сайт/место/дата, что позволяет обернуть весь процесс парсинга в приятный цикл.

In [1]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import time

In [45]:
# создаем временные индексы для цикла 
dates = pd.date_range(start='7/6/2017', end='30/4/2023')
dates = pd.Series(dates)

# переменные для удобства
moscow = 'moscow-vnukovo-airport'
spb = 'sankt-peterburg-pulkovo-airport'

# интересующие нас признаки и датафреймы для каждого города
cols = ['Ночная температура', 'Дневная температура', 'Влажность', 'Давление', 
           'Направление ветра', 'Сила ветра', 'Погодные явления']

df_moscow = pd.DataFrame(columns=cols, index=dates)
df_spb = pd.DataFrame(columns=cols, index=dates)

In [3]:
# пишем мегафункцию для парсинга 
# изначально было без try except, после ~7 ошибок из-за неравномерной структуры сайта
# добавил эти конструкции и парсинг прошел без проблем
# в конце есть time.sleep() на всякий случай, чтобы не перегружать сервер запросами

def get_values(date, place, df): 
    
    """
    Функция получает date (дату наблюдений), place (место наблюдений), 
    берет их с сайта pogoda1.ru и записывает в массиве df, где должен быть соответствующей date индекс.
    Ничего не возвращает.
    
    Записываемые наблюдения: 'Ночная температура', 'Дневная температура', 'Влажность', 'Давление', 
                           'Направление ветра', 'Сила ветра', 'Погодные явления'
                           
    Параметры:                      
    date - дата в datetime
    place - Внуково/Пулково, здесь задаются переменными moscow/spb
    df - массив для сохранения, здесь задаются переменными df_moscow/df_spb
    """

    y = date.strftime('%d-%m-%Y') # переводим дату в строку в нужном формате, чтобы подставлять в адрес 
    page_link = f'https://pogoda1.ru/{place}/{y}/' # подставляем в адрес 
    response = requests.get(page_link)
    if response.ok == True: # проверяем, пустил ли сервер на страничку; если все ок - извлекаем из html нужные данные
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # четыре контейнера obj, гле лежат нужные данные; не очень красивые преобразования строк 
        obj1 = soup.findAll(attrs={'class': 'cell-forecast-temp'})
        try:
            temp_night, temp_day = int(obj1[1].contents[0][:-1].strip()), int(obj1[3].contents[0][:-1].strip())
        except:
            temp_night, temp_day = np.nan, np.nan
        
        obj2 = soup.findAll(attrs={'class': 'weather-now-info'})
        humidity = int(obj2[1].contents[3].contents[0][:-1].strip())
        pressure = int(obj2[0].contents[3].contents[0][:3].strip())
        
        obj3 = soup.findAll(attrs={'class': 'wind-amount'})
        try:
            wind_direction = obj3[0].contents[0][:-4].split(',')[0]
            wind_velocity = int(obj3[0].contents[0][:-4].split(',')[1].strip())
        except:
            wind_direction, wind_velocity = np.nan, np.nan
        
        obj4 = soup.findAll(attrs={'class': 'weather-now-icon-img'})
        weather_cond = str(obj4[0]).split(' ')[-1].split('"')[1]
        
        # разобрались с контейнерами, теперь записываем в массив
        df.loc[date] = [temp_night, temp_day, humidity, pressure, wind_direction, wind_velocity, weather_cond]
        
    else: # для всех ошибок доступа, но практически только для 404 - когда нет данных за день
        df.loc[date] = np.full(7, np.nan)
        print(response)
        print(date)
    
    time.sleep(0.01)

In [423]:
# запускаем гуся и сохраняем результаты в csv-файлы (с соответствующми названиями будут на гитхабе); 
# индекс сохраняется в колонку, но это некритично
# этап с парсингом завершен

for date in dates:
    get_values(date, spb, df_spb)
df_spb.to_csv('weather_spb_2017_2023')

for date in dates:
    get_values(date, moscow, df_moscow)
df_moscow.to_csv('weather_moscow_2017_2023')